In [10]:
import pandas
import datetime
import pandas_datareader.data as web
import pandas_datareader
import itertools
import numpy

In [11]:
# Part 1. Gain news

In [61]:
d = './data/rex.xlsx'
data = pandas.read_excel(d)
data

,id,title,time,source,category
0,141465,"Shares rally, bond yields rise as China fuels ...",2019-08-29 22:36:30.057,REU,BUS
1,141466,"Argentine bond prices fall, country risk soars...",2019-08-29 22:36:30.057,REU,BUS
2,141467,Huawei under probe by U.S. prosecutors over ne...,2019-08-29 22:36:30.057,REU,BUS
3,141468,"As U.S. Treasuries sizzle, some bond investors...",2019-08-29 22:36:30.057,REU,BUS
4,141469,"China trade comments lift Wall Street as tech,...",2019-08-29 22:36:30.057,REU,BUS
...,...,...,...,...,...
9620,406845,U.S. unemployment rate hits 3.5%; job growth m...,2019-10-05 06:04:32.200,REU,BUS
9621,406846,Instant View: U.S. September payrolls lower th...,2019-10-05 06:04:32.200,REU,BUS
9622,406895,Hong Kong metro stays shut after night of viol...,2019-10-05 06:12:19.239,REU,WRL
9623,406995,UPDATE 3-PayPal becomes first member to exit F...,2019-10-05 06:28:54.713,REU,CMP


In [62]:
# Part 2. Gain quotes

In [63]:
k = 'C:/Users/MainUser/Desktop/AlphaVantage.txt'
crs = open(k, "r")
for columns in ( raw.strip().split() for raw in crs ): 
    api_key = columns[0]


In [64]:
"""
IDK what's this, they just came from somewhere...

for:   dayXnewstitle, ticker, news_horizont (how much it lasted from the news' release), effect_horizont (the horizon we consider effect on) 
spec (most general):  affected_target ~ (self lags), (related targets from other observations), (related objects), (indicators), (macro)

dimension0 (kinds): yield itself, yield variance, specific properties of it's process

dimension1 (regime):
    group
        model -- by group
        (groups: stocks, bonds, etfs, indicators, goods)
    individual
        model -- by quote

dimension2 (variety of fields):
    1 = self only
    2 = 1 + others from its config observ
    3 = 1 + news
    4 = 1 + others from its config observ + news
"""
pass

In [65]:
target_quotes = ['MSFT']
max_newstitle_lag = 3    # = news_horizont
max_quotes_lag = 3       # for lags
effect_horizon = 1

newstitles = data[['id', 'time', 'title']]
lag_markers = list(itertools.product(newstitles['id'].values, numpy.array(numpy.arange(max_newstitle_lag - 1)) + 1 ))
lag_markers = pandas.DataFrame(data=lag_markers, columns=['id', 'lag'])

newstitles = newstitles.merge(right=lag_markers, left_on=['id'], right_on=['id'])

def add_lag(x):
    a = x['time'] + pandas.DateOffset(days=x['lag'])
    return datetime.date(a.year, a.month, a.day)

newstitles['target_date'] = newstitles.apply(func=add_lag, axis=1)
#newstitles = newstitles.set_index(keys=['target_date'])
newstitles

,id,time,title,lag,target_date
0,141465,2019-08-29 22:36:30.057,"Shares rally, bond yields rise as China fuels ...",1,2019-08-30
1,141465,2019-08-29 22:36:30.057,"Shares rally, bond yields rise as China fuels ...",2,2019-08-31
2,141466,2019-08-29 22:36:30.057,"Argentine bond prices fall, country risk soars...",1,2019-08-30
3,141466,2019-08-29 22:36:30.057,"Argentine bond prices fall, country risk soars...",2,2019-08-31
4,141467,2019-08-29 22:36:30.057,Huawei under probe by U.S. prosecutors over ne...,1,2019-08-30
...,...,...,...,...,...
19245,406895,2019-10-05 06:12:19.239,Hong Kong metro stays shut after night of viol...,2,2019-10-07
19246,406995,2019-10-05 06:28:54.713,UPDATE 3-PayPal becomes first member to exit F...,1,2019-10-06
19247,406995,2019-10-05 06:28:54.713,UPDATE 3-PayPal becomes first member to exit F...,2,2019-10-07
19248,407015,2019-10-05 06:28:58.138,PayPal becomes first member to exit Facebook's...,1,2019-10-06


In [66]:
beginning_date, ending_date = newstitles['target_date'].min() - datetime.timedelta(days=(max_quotes_lag + effect_horizon)), newstitles['target_date'].max()
beginning_date, ending_date

(datetime.date(2019, 8, 26), datetime.date(2019, 10, 7))

In [86]:
the_batches = []

for target_quote in target_quotes:
    f = pandas_datareader.av.time_series.AVTimeSeriesReader(symbols=target_quote, function='TIME_SERIES_DAILY', 
                                                        start=beginning_date, end=ending_date, 
                                                        #retry_count=3, pause=0.1, session=None, chunksize=25, 
                                                        api_key=api_key)

    ff = f.read()
    ff['ticker'] = target_quote
    the_batches.append(ff)
quotes_data = pandas.concat(the_batches)
quotes_data = quotes_data.sort_index(ascending=True)
quotes_data

Hi guys, I'm here


,open,high,low,close,volume,ticker
2019-08-26,134.99,135.5600,133.9000,135.45,20325271,MSFT
2019-08-27,136.39,136.7200,134.6600,135.74,23115635,MSFT
2019-08-28,134.88,135.7600,133.5500,135.56,17343042,MSFT
2019-08-29,137.25,138.4400,136.9100,138.12,20179716,MSFT
2019-08-30,139.15,139.1800,136.2700,137.86,23946123,MSFT
2019-09-03,136.61,137.2000,135.7000,136.04,18880773,MSFT
2019-09-04,137.30,137.6900,136.4800,137.63,18015310,MSFT
2019-09-05,139.11,140.3837,138.7600,140.05,26119520,MSFT
2019-09-06,140.03,140.1800,138.2000,139.10,20824504,MSFT
2019-09-09,139.59,139.7500,136.4600,137.52,25773932,MSFT


In [87]:
quotes_data.shape

(30, 6)

In [88]:
def lag(array, names, ex, upon=None, exactly=None, appx='LAG'):
    if upon is not None:
        result, rname = [], []
        arra = array[:, [x not in ex for x in names]]
        for j in range((upon + 1)):
            re = numpy.roll(arra, shift=j, axis=0)
            re[:j] = numpy.nan
            result.append(re)
            rname.append([x + '_{}{}'.format(appx, j) for x in names if x not in ex])
        return numpy.concatenate((numpy.concatenate(result, axis=1), array[:, [x in ex for x in names]]), axis=1), numpy.concatenate((numpy.concatenate(rname), ex))
    if exactly is not None:
        result, rname = [], []
        arra = array[:, [x not in ex for x in names]]
        for j in [0, exactly]:
            re = numpy.roll(arra, shift=j, axis=0)
            re[:j] = numpy.nan
            result.append(re)
            rname.append([x + '_{}{}'.format(appx, j) for x in names if x not in ex])
        return numpy.concatenate((numpy.concatenate(result, axis=1), array[:, [x in ex for x in names]]), axis=1), numpy.concatenate((numpy.concatenate(rname), ex))

In [89]:
quotes_data_lagged_values, quotes_data_lagged_columns = lag(array=quotes_data.values, names=quotes_data.columns.values, exactly=effect_horizon, appx='hori', ex=['ticker'])
quotes_data = pandas.DataFrame(data=quotes_data_lagged_values, index=quotes_data.index.values, columns=quotes_data_lagged_columns)
quotes_data

,open_hori0,high_hori0,low_hori0,close_hori0,volume_hori0,open_hori1,high_hori1,low_hori1,close_hori1,volume_hori1,ticker
2019-08-26,134.99,135.56,133.9,135.45,20325271,NaN,NaN,NaN,NaN,NaN,MSFT
2019-08-27,136.39,136.72,134.66,135.74,23115635,134.99,135.56,133.9,135.45,20325271,MSFT
2019-08-28,134.88,135.76,133.55,135.56,17343042,136.39,136.72,134.66,135.74,23115635,MSFT
2019-08-29,137.25,138.44,136.91,138.12,20179716,134.88,135.76,133.55,135.56,17343042,MSFT
2019-08-30,139.15,139.18,136.27,137.86,23946123,137.25,138.44,136.91,138.12,20179716,MSFT
2019-09-03,136.61,137.2,135.7,136.04,18880773,139.15,139.18,136.27,137.86,23946123,MSFT
2019-09-04,137.3,137.69,136.48,137.63,18015310,136.61,137.2,135.7,136.04,18880773,MSFT
2019-09-05,139.11,140.384,138.76,140.05,26119520,137.3,137.69,136.48,137.63,18015310,MSFT
2019-09-06,140.03,140.18,138.2,139.1,20824504,139.11,140.384,138.76,140.05,26119520,MSFT
2019-09-09,139.59,139.75,136.46,137.52,25773932,140.03,140.18,138.2,139.1,20824504,MSFT


In [90]:
quotes_data.columns.values

array(['open_hori0', 'high_hori0', 'low_hori0', 'close_hori0',
       'volume_hori0', 'open_hori1', 'high_hori1', 'low_hori1',
       'close_hori1', 'volume_hori1', 'ticker'], dtype=object)

In [91]:
quotes_data.columns.values[:5], quotes_data.columns.values[5:10]

(array(['open_hori0', 'high_hori0', 'low_hori0', 'close_hori0',
        'volume_hori0'], dtype=object),
 array(['open_hori1', 'high_hori1', 'low_hori1', 'close_hori1',
        'volume_hori1'], dtype=object))

In [92]:
j = 0
quotes_data.columns.values[j], quotes_data.columns.values[5 + j]

('open_hori0', 'open_hori1')

In [93]:
nn = 5
for j in range(nn):
    quotes_data.iloc[:, 5 + j] = quotes_data.iloc[:, 5 + j] / quotes_data.iloc[:, j] - 1
quotes_data = quotes_data.drop(columns=[quotes_data.columns.values[j] for j in range(nn)])
quotes_data

,open_hori1,high_hori1,low_hori1,close_hori1,volume_hori1,ticker
2019-08-26,NaN,NaN,NaN,NaN,NaN,MSFT
2019-08-27,-0.0102647,-0.00848449,-0.00564384,-0.00213644,-0.120713,MSFT
2019-08-28,0.0111951,0.0070713,0.00831149,0.00132783,0.332848,MSFT
2019-08-29,-0.0172678,-0.0193586,-0.0245417,-0.0185346,-0.140571,MSFT
2019-08-30,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,MSFT
2019-09-03,0.0185931,0.0144315,0.00420044,0.0133784,0.268281,MSFT
2019-09-04,-0.00502549,-0.00355872,-0.00571512,-0.0115527,0.0480404,MSFT
2019-09-05,-0.0130113,-0.0191881,-0.0164312,-0.0172795,-0.310274,MSFT
2019-09-06,-0.00657002,0.00145313,0.0040521,0.00682962,0.254269,MSFT
2019-09-09,0.00315209,0.00307692,0.012751,0.0114892,-0.192032,MSFT


In [95]:
quotes_data = quotes_data.dropna()
quotes_data

,open_hori1,high_hori1,low_hori1,close_hori1,volume_hori1,ticker
2019-08-27,-0.0102647,-0.00848449,-0.00564384,-0.00213644,-0.120713,MSFT
2019-08-28,0.0111951,0.0070713,0.00831149,0.00132783,0.332848,MSFT
2019-08-29,-0.0172678,-0.0193586,-0.0245417,-0.0185346,-0.140571,MSFT
2019-08-30,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,MSFT
2019-09-03,0.0185931,0.0144315,0.00420044,0.0133784,0.268281,MSFT
2019-09-04,-0.00502549,-0.00355872,-0.00571512,-0.0115527,0.0480404,MSFT
2019-09-05,-0.0130113,-0.0191881,-0.0164312,-0.0172795,-0.310274,MSFT
2019-09-06,-0.00657002,0.00145313,0.0040521,0.00682962,0.254269,MSFT
2019-09-09,0.00315209,0.00307692,0.012751,0.0114892,-0.192032,MSFT
2019-09-10,0.0203947,0.020921,0.0144971,0.010582,-0.108273,MSFT


In [97]:
quotes_data_lagged_values, quotes_data_lagged_columns = lag(array=quotes_data.values, names=quotes_data.columns.values, upon=max_quotes_lag, ex=['ticker'])
quotes_data_lagged = pandas.DataFrame(data=quotes_data_lagged_values, index=quotes_data.index.values, columns=quotes_data_lagged_columns)
quotes_data_lagged

,open_hori1_LAG0,high_hori1_LAG0,low_hori1_LAG0,close_hori1_LAG0,volume_hori1_LAG0,open_hori1_LAG1,high_hori1_LAG1,low_hori1_LAG1,close_hori1_LAG1,volume_hori1_LAG1,...,high_hori1_LAG2,low_hori1_LAG2,close_hori1_LAG2,volume_hori1_LAG2,open_hori1_LAG3,high_hori1_LAG3,low_hori1_LAG3,close_hori1_LAG3,volume_hori1_LAG3,ticker
2019-08-27,-0.0102647,-0.00848449,-0.00564384,-0.00213644,-0.120713,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSFT
2019-08-28,0.0111951,0.0070713,0.00831149,0.00132783,0.332848,-0.0102647,-0.00848449,-0.00564384,-0.00213644,-0.120713,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSFT
2019-08-29,-0.0172678,-0.0193586,-0.0245417,-0.0185346,-0.140571,0.0111951,0.0070713,0.00831149,0.00132783,0.332848,...,-0.00848449,-0.00564384,-0.00213644,-0.120713,NaN,NaN,NaN,NaN,NaN,MSFT
2019-08-30,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,-0.140571,...,0.0070713,0.00831149,0.00132783,0.332848,-0.0102647,-0.00848449,-0.00564384,-0.00213644,-0.120713,MSFT
2019-09-03,0.0185931,0.0144315,0.00420044,0.0133784,0.268281,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,...,-0.0193586,-0.0245417,-0.0185346,-0.140571,0.0111951,0.0070713,0.00831149,0.00132783,0.332848,MSFT
2019-09-04,-0.00502549,-0.00355872,-0.00571512,-0.0115527,0.0480404,0.0185931,0.0144315,0.00420044,0.0133784,0.268281,...,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,-0.140571,MSFT
2019-09-05,-0.0130113,-0.0191881,-0.0164312,-0.0172795,-0.310274,-0.00502549,-0.00355872,-0.00571512,-0.0115527,0.0480404,...,0.0144315,0.00420044,0.0133784,0.268281,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,MSFT
2019-09-06,-0.00657002,0.00145313,0.0040521,0.00682962,0.254269,-0.0130113,-0.0191881,-0.0164312,-0.0172795,-0.310274,...,-0.00355872,-0.00571512,-0.0115527,0.0480404,0.0185931,0.0144315,0.00420044,0.0133784,0.268281,MSFT
2019-09-09,0.00315209,0.00307692,0.012751,0.0114892,-0.192032,-0.00657002,0.00145313,0.0040521,0.00682962,0.254269,...,-0.0191881,-0.0164312,-0.0172795,-0.310274,-0.00502549,-0.00355872,-0.00571512,-0.0115527,0.0480404,MSFT
2019-09-10,0.0203947,0.020921,0.0144971,0.010582,-0.108273,0.00315209,0.00307692,0.012751,0.0114892,-0.192032,...,0.00145313,0.0040521,0.00682962,0.254269,-0.0130113,-0.0191881,-0.0164312,-0.0172795,-0.310274,MSFT


In [98]:
quotes_data_lagged = quotes_data_lagged.dropna()
quotes_data_lagged.head(5)

,open_hori1_LAG0,high_hori1_LAG0,low_hori1_LAG0,close_hori1_LAG0,volume_hori1_LAG0,open_hori1_LAG1,high_hori1_LAG1,low_hori1_LAG1,close_hori1_LAG1,volume_hori1_LAG1,...,high_hori1_LAG2,low_hori1_LAG2,close_hori1_LAG2,volume_hori1_LAG2,open_hori1_LAG3,high_hori1_LAG3,low_hori1_LAG3,close_hori1_LAG3,volume_hori1_LAG3,ticker
2019-08-30,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,-0.140571,...,0.0070713,0.00831149,0.00132783,0.332848,-0.0102647,-0.00848449,-0.00564384,-0.00213644,-0.120713,MSFT
2019-09-03,0.0185931,0.0144315,0.00420044,0.0133784,0.268281,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,...,-0.0193586,-0.0245417,-0.0185346,-0.140571,0.0111951,0.0070713,0.00831149,0.00132783,0.332848,MSFT
2019-09-04,-0.00502549,-0.00355872,-0.00571512,-0.0115527,0.0480404,0.0185931,0.0144315,0.00420044,0.0133784,0.268281,...,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,-0.140571,MSFT
2019-09-05,-0.0130113,-0.0191881,-0.0164312,-0.0172795,-0.310274,-0.00502549,-0.00355872,-0.00571512,-0.0115527,0.0480404,...,0.0144315,0.00420044,0.0133784,0.268281,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,MSFT
2019-09-06,-0.00657002,0.00145313,0.0040521,0.00682962,0.254269,-0.0130113,-0.0191881,-0.0164312,-0.0172795,-0.310274,...,-0.00355872,-0.00571512,-0.0115527,0.0480404,0.0185931,0.0144315,0.00420044,0.0133784,0.268281,MSFT


In [99]:
quotes_data_lagged = quotes_data_lagged.reset_index()
quotes_data_lagged.head(5)

,index,open_hori1_LAG0,high_hori1_LAG0,low_hori1_LAG0,close_hori1_LAG0,volume_hori1_LAG0,open_hori1_LAG1,high_hori1_LAG1,low_hori1_LAG1,close_hori1_LAG1,...,high_hori1_LAG2,low_hori1_LAG2,close_hori1_LAG2,volume_hori1_LAG2,open_hori1_LAG3,high_hori1_LAG3,low_hori1_LAG3,close_hori1_LAG3,volume_hori1_LAG3,ticker
0,2019-08-30,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,...,0.0070713,0.00831149,0.00132783,0.332848,-0.0102647,-0.00848449,-0.00564384,-0.00213644,-0.120713,MSFT
1,2019-09-03,0.0185931,0.0144315,0.00420044,0.0133784,0.268281,-0.0136543,-0.00531686,0.00469656,0.00188597,...,-0.0193586,-0.0245417,-0.0185346,-0.140571,0.0111951,0.0070713,0.00831149,0.00132783,0.332848,MSFT
2,2019-09-04,-0.00502549,-0.00355872,-0.00571512,-0.0115527,0.0480404,0.0185931,0.0144315,0.00420044,0.0133784,...,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,-0.140571,MSFT
3,2019-09-05,-0.0130113,-0.0191881,-0.0164312,-0.0172795,-0.310274,-0.00502549,-0.00355872,-0.00571512,-0.0115527,...,0.0144315,0.00420044,0.0133784,0.268281,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,MSFT
4,2019-09-06,-0.00657002,0.00145313,0.0040521,0.00682962,0.254269,-0.0130113,-0.0191881,-0.0164312,-0.0172795,...,-0.00355872,-0.00571512,-0.0115527,0.0480404,0.0185931,0.0144315,0.00420044,0.0133784,0.268281,MSFT


In [100]:
def to_date(x):
    return datetime.date(int(x[:4]), int(x[5:7]), int(x[8:]))

quotes_data_lagged['index'] = quotes_data_lagged['index'].apply(func=to_date)
quotes_data_lagged.head(5)

,index,open_hori1_LAG0,high_hori1_LAG0,low_hori1_LAG0,close_hori1_LAG0,volume_hori1_LAG0,open_hori1_LAG1,high_hori1_LAG1,low_hori1_LAG1,close_hori1_LAG1,...,high_hori1_LAG2,low_hori1_LAG2,close_hori1_LAG2,volume_hori1_LAG2,open_hori1_LAG3,high_hori1_LAG3,low_hori1_LAG3,close_hori1_LAG3,volume_hori1_LAG3,ticker
0,2019-08-30,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,...,0.0070713,0.00831149,0.00132783,0.332848,-0.0102647,-0.00848449,-0.00564384,-0.00213644,-0.120713,MSFT
1,2019-09-03,0.0185931,0.0144315,0.00420044,0.0133784,0.268281,-0.0136543,-0.00531686,0.00469656,0.00188597,...,-0.0193586,-0.0245417,-0.0185346,-0.140571,0.0111951,0.0070713,0.00831149,0.00132783,0.332848,MSFT
2,2019-09-04,-0.00502549,-0.00355872,-0.00571512,-0.0115527,0.0480404,0.0185931,0.0144315,0.00420044,0.0133784,...,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,-0.140571,MSFT
3,2019-09-05,-0.0130113,-0.0191881,-0.0164312,-0.0172795,-0.310274,-0.00502549,-0.00355872,-0.00571512,-0.0115527,...,0.0144315,0.00420044,0.0133784,0.268281,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,MSFT
4,2019-09-06,-0.00657002,0.00145313,0.0040521,0.00682962,0.254269,-0.0130113,-0.0191881,-0.0164312,-0.0172795,...,-0.00355872,-0.00571512,-0.0115527,0.0480404,0.0185931,0.0144315,0.00420044,0.0133784,0.268281,MSFT


In [101]:
newstitles.head(5)

,id,time,title,lag,target_date
0,141465,2019-08-29 22:36:30.057,"Shares rally, bond yields rise as China fuels ...",1,2019-08-30
1,141465,2019-08-29 22:36:30.057,"Shares rally, bond yields rise as China fuels ...",2,2019-08-31
2,141466,2019-08-29 22:36:30.057,"Argentine bond prices fall, country risk soars...",1,2019-08-30
3,141466,2019-08-29 22:36:30.057,"Argentine bond prices fall, country risk soars...",2,2019-08-31
4,141467,2019-08-29 22:36:30.057,Huawei under probe by U.S. prosecutors over ne...,1,2019-08-30


In [102]:
#the_data = newstitles.merge(right=quotes_data_lagged, left_index=True, right_index=True)
the_data = quotes_data_lagged.merge(right=newstitles, left_on='index', right_on='target_date')
the_data

,index,open_hori1_LAG0,high_hori1_LAG0,low_hori1_LAG0,close_hori1_LAG0,volume_hori1_LAG0,open_hori1_LAG1,high_hori1_LAG1,low_hori1_LAG1,close_hori1_LAG1,...,high_hori1_LAG3,low_hori1_LAG3,close_hori1_LAG3,volume_hori1_LAG3,ticker,id,time,title,lag,target_date
0,2019-08-30,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,...,-0.00848449,-0.00564384,-0.00213644,-0.120713,MSFT,141465,2019-08-29 22:36:30.057,"Shares rally, bond yields rise as China fuels ...",1,2019-08-30
1,2019-08-30,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,...,-0.00848449,-0.00564384,-0.00213644,-0.120713,MSFT,141466,2019-08-29 22:36:30.057,"Argentine bond prices fall, country risk soars...",1,2019-08-30
2,2019-08-30,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,...,-0.00848449,-0.00564384,-0.00213644,-0.120713,MSFT,141467,2019-08-29 22:36:30.057,Huawei under probe by U.S. prosecutors over ne...,1,2019-08-30
3,2019-08-30,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,...,-0.00848449,-0.00564384,-0.00213644,-0.120713,MSFT,141468,2019-08-29 22:36:30.057,"As U.S. Treasuries sizzle, some bond investors...",1,2019-08-30
4,2019-08-30,-0.0136543,-0.00531686,0.00469656,0.00188597,-0.157287,-0.0172678,-0.0193586,-0.0245417,-0.0185346,...,-0.00848449,-0.00564384,-0.00213644,-0.120713,MSFT,141469,2019-08-29 22:36:30.057,"China trade comments lift Wall Street as tech,...",1,2019-08-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10808,2019-10-07,-0.00284381,0.000506586,-0.00437892,0.00729288,0.436006,-0.0131627,-0.0108499,-0.023457,-0.0133217,...,0.028452,0.0256034,0.0179725,-0.294354,MSFT,406845,2019-10-05 06:04:32.200,U.S. unemployment rate hits 3.5%; job growth m...,2,2019-10-07
10809,2019-10-07,-0.00284381,0.000506586,-0.00437892,0.00729288,0.436006,-0.0131627,-0.0108499,-0.023457,-0.0133217,...,0.028452,0.0256034,0.0179725,-0.294354,MSFT,406846,2019-10-05 06:04:32.200,Instant View: U.S. September payrolls lower th...,2,2019-10-07
10810,2019-10-07,-0.00284381,0.000506586,-0.00437892,0.00729288,0.436006,-0.0131627,-0.0108499,-0.023457,-0.0133217,...,0.028452,0.0256034,0.0179725,-0.294354,MSFT,406895,2019-10-05 06:12:19.239,Hong Kong metro stays shut after night of viol...,2,2019-10-07
10811,2019-10-07,-0.00284381,0.000506586,-0.00437892,0.00729288,0.436006,-0.0131627,-0.0108499,-0.023457,-0.0133217,...,0.028452,0.0256034,0.0179725,-0.294354,MSFT,406995,2019-10-05 06:28:54.713,UPDATE 3-PayPal becomes first member to exit F...,2,2019-10-07


In [6]:
a = pandas.DataFrame(data={'a': [1,2,3,4], 'b': [0.1, 0.2, 0.3, 0.4]})
a

,a,b
0,1,0.1
1,2,0.2
2,3,0.3
3,4,0.4


In [7]:
b = pandas.DataFrame(data={'a': [1,2,2,2,3,3,3,4], 'b': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]})
b

,a,b
0,1,0.1
1,2,0.2
2,2,0.3
3,2,0.4
4,3,0.5
5,3,0.6
6,3,0.7
7,4,0.8


In [9]:
a.merge(right=b, left_on=['a'], right_on=['a'])

,a,b_x,b_y
0,1,0.1,0.1
1,2,0.2,0.2
2,2,0.2,0.3
3,2,0.2,0.4
4,3,0.3,0.5
5,3,0.3,0.6
6,3,0.3,0.7
7,4,0.4,0.8


In [ ]:

merge (quotes to news: quotes_date to news_target_date)


    

In [6]:
target_quote = "AAPL"

lmao_bottom_text = data['time'].min()
lmao_top_text = data['time'].max()

"""
f = web.get_quote_av(["AAPL", "TSLA"], api_key=api_key)
"""
"""
f = web.DataReader(target_quote, "av-intraday",
                   #start=lmao_bottom_text, end=lmao_top_text, 
                   start=datetime(2017, 2, 9), end=datetime(2017, 5, 24),
                   api_key=api_key)
"""
"""
f = pandas_datareader.av.time_series.AVTimeSeriesReader(symbols=target_quote, function='TIME_SERIES_INTRADAY', 
                                                        #start=lmao_bottom_text, end=lmao_top_text, 
                                                        #start='2019-08-29', end='2019-10-05',
                                                        start='2020-01-01', end='2020-01-10',
                                                        #retry_count=3, pause=0.1, session=None, chunksize=25, 
                                                        api_key=api_key)
"""


f = pandas_datareader.av.time_series.AVTimeSeriesReader(symbols=target_quote, function='TIME_SERIES_DAILY', 
                                                        start=lmao_bottom_text, end=lmao_top_text, 
                                                        #retry_count=3, pause=0.1, session=None, chunksize=25, 
                                                        api_key=api_key)


f.read()

Hi guys, I'm here


,open,high,low,close,volume
2019-08-29,208.500,209.32,206.6550,209.010,21007652
2019-08-30,210.160,210.45,207.2000,208.740,21162561
2019-09-03,206.430,206.98,204.2200,205.700,20059574
2019-09-04,208.390,209.48,207.3200,209.190,19216820
2019-09-05,212.000,213.97,211.5100,213.280,23946984
2019-09-06,214.050,214.42,212.5100,213.260,19362294
2019-09-09,214.840,216.44,211.0700,214.170,27309401
2019-09-10,213.860,216.78,211.7100,216.700,31777931
2019-09-11,218.070,223.71,217.7300,223.590,44289646
2019-09-12,224.800,226.42,222.8600,223.085,32226669


In [7]:
lmao_bottom_text

Timestamp('2019-08-29 22:36:30.057000')

In [8]:
lmao_top_text

Timestamp('2019-10-05 06:28:58.138000')